In [28]:
import pandas as pd
import requests
import numpy as np
from requests_html import HTML,HTMLSession
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import choice
from sklearn.neighbors import NearestNeighbors
import math
import time

ModuleNotFoundError: No module named 'git'

NameError: name 'git' is not defined

In [2]:
#ideas: Take the names from the json file, in order to make the enumeration funny

In [10]:
olga=pd.read_csv('olga.csv')
#olga[olga.partition=='train'].count()   #train 0-14138, #val 14139-15905, #test 15906-17673
olga.musicbrainz_id[11902]

'846be3c9-5f94-46ab-97b9-531335dd3658'

In [11]:
class DatasetOlga():
    
    def __init__(self,olga):
        self.olga=olga
        self.mb=olga.musicbrainz_id
        self.artists={} #Needed for obtaining the mapping from musicbrainz to the allmusic ids
        self.l=len(self.mb)
        self.d={}       #Needed for obtaining a dict. where keys are artists, and values are the artists similar to them, based on self.artists
        self.NI={}      #Dict. that will contain the artist's features
    def get_mapping(self,i):
        response = requests.get(f'https://musicbrainz.org/ws/2/artist/{str(self.mb[i])}?inc=url-rels&fmt=json')
        if response.ok:
            data = response.json()
            refs = [r['url']['resource'] for r in data['relations'] if r['type'] == 'allmusic']        
            return refs[0] if len(refs) != 0 else "Not found"

        

    def get_mappingList(self,init,end,increm=500):
        Lmusicbrainz_id=self.mb[init:end]
        length=len(Lmusicbrainz_id)
        c=0
        for i in range(len(Lmusicbrainz_id)):
            mapp=self.get_mapping(i)
            if mapp==None:
                while mapp==None:
                    mapp=self.get_mapping(i)
                    
            if mapp!="Not found":   #Some of the ids has not a respective allmusic id, so we lose that information
                mapp=str(mapp)      #Mapp are strings of links
                key=mapp[-12:]
                self.artists[key]=i
            c+=1
            if c%increm==0 or c==30:
                    print("{}/{} artists were processed".format(c,length))
                    
            
        #self.save_data(self.artists,'MsbMapped1.json')  #We do save the Artists Ids map
            
        return self.artists
    
    
    def get_GraphDict(self,name='MsbMapped1.json',increm=500):
        session=HTMLSession()
        c=0 #Counter
        artID=self.load_data(name) #We load the mapped artists (between MusicBrainz Ids, and AllMusic Ids)
        length=len(artID.keys())
        for k in artID.keys(): #dict of mapped mbids, this has to be computed before from getmapping
            if k!=None:
                url='https://www.allmusic.com/artist/'+ k+ '/related' #k is just the code
                r=session.get(url)
                sess=r.html.find('body',first=True)
                div=sess.find('.overflow-container')
                divn=div[0]
                divn=divn.find('.content-container')
                divn=divn[0]
                divn=divn.find('.content')
                divn=divn[0]
                divn=divn.find('section',first=True)
                if divn==None:
                    self.d[artID[k]]=[] #That artist has not related artists (or we have missing informations)
                    continue
                artists=divn.find('li')
                artistL=[]


                for i in range(len(artists)):
                    art=artists[i]
                    art=art.find('a')
                    link=list(art[0].absolute_links)[0] #Absolute_link return a one-element set, that we convert into a list and
                    link=str(link)[-12:] #we get its code
                    if link in artID.keys(): #g is the dict of all the mapped musicbrainz_ids
                        artistL.append(self.artists[link])
                self.d[artID[k]]=artistL
                c+=1
                if c%increm==0 or c==30:
                    print("{}/{} artists were processed".format(c,length))
        #self.save_data(self.d,'graphSimilarities1.json') #Here we save the connection amongst the artist, obtained with this method
        print("Done...")
        return self.d
    
    def save_data(self,dicti,name):
        jfile = open(name, "w")
        jfile = json.dump(dicti, jfile)
    
    def load_data(self,name):
        jfile = open(name, "r")
        dicti = json.load(jfile)
        return dicti


In [14]:
d=DatasetOlga(olga)
d.get_mapping(11902)

'https://www.allmusic.com/artist/mn0000678420'

In [5]:
n_features=2613
class Graph():  #The purpose of this class is to construct the graph of artists, in particular the Adjacency matrix A, and the  
                # node features tensor X
        
    def __init__(self,mapfile,gfile):
        self.mfile=self.load_data(mapfile)
        self.gfile=self.load_data(gfile)
        self.A=torch.zeros((len(self.mfile),len(self.mfile)))
        self.X=torch.zeros((n_features,len(self.mfile)))
        self.ord=sorted(list(map(int,self.gfile.keys())))
        self.enc1={}
        self.enc2={}
        
    def encoding1(self):   #From ordered to unordered, Dict are not ordered data structures, so is better to order them before
        for k in range(len(self.mfile)): #This encoding is used to get the Instance matrix
            self.enc1[k]=self.ord[k]
        return self.enc1
    
    def encoding2(self):   #From unordered to ordered,  From real number, to ordered one.
        for k in range(len(self.mfile)): #This encoding is used to get the Adjacency matrix
            self.enc2[self.ord[k]]=k
        return self.enc2
    
    def get_instance(self,instances,df=False):#We take the features centroid, obtained from 25 track from artists discographies.
        X=np.load(instances)
        X=torch.from_numpy(X).requires_grad_(True)  #We take the allmusicIDs, which contain the key of the artists for which we don't 
        c=0                    # have lost the informations
        enc=self.encoding1()
        for k in self.mfile:
            z=enc[c]
            self.X[:,c]=X[z] 
            c+=1
        return self.X
    
    def get_adjacency(self,symmetry=False,df=False):  #The hypothesis could be either a symmetric matrix (paper), or not.
        enc=self.encoding2()
        for k in self.gfile:
            c1=enc[int(k)]
            for j in self.gfile[k]:
                c2=enc[int(j)]
                if self.A[c2,c1]==1 and symmetry==True:
                    continue
                self.A[c1,c2]=1
                if symmetry:
                    self.A[c2,c1]=1

            
        return self.A
    
    
    def load_data(self,name):
        jfile = open(name, "r")
        dicti = json.load(jfile)
        return dicti
    
g=Graph('MsbMapped.json','graphSimilarities.json')
X1=g.get_instance('acousticbrainz.npy')
A1=g.get_adjacency(symmetry=True)


In [6]:
l=list(range(0,9021+1))
l[-1]

9021

In [22]:
#Here we have some hyperparamters, and also the list of ordered artists, and their indices with respect their own set (train,val,test).
outf=[2613,512,128,32,100] #Train: 0:9021, #Val: 9022:10189, #Test: 10190:11260
train_=list(range(0,9021+1)) #Train set, without val
train=list(range(0,10189+1)) #Train set, with val
val=list(range(9022,10189+1))
test=list(range(10190,11260+1))
KNN=200
device=torch.device('cuda')
class GraphSAGE(nn.Module):
    
    def __init__(self,X,A):
        super(GraphSAGE,self).__init__()
        self.A=A #Tensors version of adjacency matrix and Instances
        self.X=X    
        self.V={}  #In this dictionary we insert the tracing of a certain mini_batch, needed for the forward step
        self.l11=nn.Linear(outf[0],1024)
        self.l12=nn.Linear(3637,outf[1])
        self.l21=nn.Linear(outf[1],256)
        self.l22=nn.Linear(768,outf[2])
        self.l31=nn.Linear(outf[2],64)
        self.l32=nn.Linear(192,outf[3])
        self.FC11=nn.Linear(outf[1],256)
        self.FC12=nn.Linear(outf[2],256)
        self.FC13=nn.Linear(outf[3],256)
        
        self.FC2=nn.Linear(256,256)
        self.out=nn.Linear(256,100) #final output layer
        
        
    
    def forward(self,V,L):
        K=L+1  #We add a number, because if we have three layer, we need to count also the batch
        self.V[K]=set(V)
        for k in range(K-1,0,-1):
            d=set()
            for idx in self.V[k+1]: 
                
                d=d.union(self.get_n(idx))
                
            self.V[k]=d
        
        Es=self.select(self.X,set(),self.V[1])
        for k in range(0,K-1): #k starts from 0, 0 is associated with the first layer, 1 with the second and so on....
            #print("Graph convolutional layer n° {}, processing...".format(k+1))
            t=self.tfunc(outf[k],Es,self.V[k+1]) #tfunc is a matrix that has 0 for the column outside the mini-batch sets
            Esn=self.select(t,set(),self.V[k+2]) #and has the transformed vectors for the columns that belongs to the mini_batch set
            An=self.select(self.A,self.V[k+1],self.V[k+2]) #We do select either the vectors from X, and from A
            if k==0:
                N=torch.mm(torch.transpose(F.elu(self.l11(torch.transpose(Es,0,1))),0,1),An)
                
                Es=F.elu(self.l12(torch.transpose(torch.cat((N,Esn)),0,1)))
            if k==1:
                N=torch.mm(torch.transpose(F.elu(self.l21(torch.transpose(Es,0,1))),0,1),An)
                
                Es=F.elu(self.l22(torch.transpose(torch.cat((N,Esn)),0,1)))
            if k==2:
                N=torch.mm(torch.transpose(F.elu(self.l31(torch.transpose(Es,0,1))),0,1),An)
                
                Es=F.elu(self.l32(torch.transpose(torch.cat((N,Esn)),0,1)))
            
            Es=torch.transpose(F.normalize(Es,dim=0),0,1)
        if L==1:                                         #There are different input dimensions, because they depend on the 
            Es=F.elu(self.FC11(torch.transpose(Es,0,1))) #output dimension returned by the GCN
        elif L==2:
            Es=F.elu(self.FC12(torch.transpose(Es,0,1)))
        elif L==3:
            Es=F.elu(self.FC13(torch.transpose(Es,0,1)))
        Es=F.elu(self.FC2(Es))
        Es=torch.transpose(self.out(Es),0,1) #Final linear layer that represent the obtained embedded space.
#         print(Es)
#         print(Es.shape)
        return Es
            
    def get_n(self,idx):    #This function is the neighbors function. Given a batch index we get its neighborhood.
        t=torch.nonzero(self.A[idx])
        s=set()
        
        for k in t:
            if t.shape[0]!=0:
                s.add(k.item())
        s.add(idx)     
                
        return s
    def select(self,mat,row,col):  #Given a set of indices for rows or column or both, we get the respective elements.
        col=sorted(list(col))      #This is applied when we get the t matrix.
        
        c=0
        if row==set():
            ma=torch.zeros((mat.shape[0],len(col)))
            for k in col:
                ma[:,c]=mat[:,k]
                c+=1
            return ma
        else:
            row=torch.tensor(sorted(list(row)))
            col=torch.tensor(col)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
    def tfunc(self,n_feat,es,V):               #This is the t function, and it is previously described
        t=torch.zeros((n_feat,self.X.shape[1]))
        V=sorted(list(V))
        c=0
        for k in V:
            t[:,k]=es[:,c]
            c+=1
        return t
    def tfunc2(self,n_feat,es,V,prev):               #This is the t function, and it is previously described
        #t=torch.zeros((n_feat,self.X.shape[1]))
        V=sorted(list(V))
        c=0
        for k in V:
            prev[:,k]=es[:,c]
            c+=1
        return prev
    def mini_batches(self,indices,bs=32): #This function generates a list of minibatches, of size bs
        indicesN=indices.copy()
        mbList=[] #Lists of lists of mini_batches indices 
        while len(indicesN)!=0:
            mb=set()  #Inner list, with the indices of a particular mini_batch
            while len(mb)<bs:
                if len(indicesN)==0:
                    mbList.append(mb)
                    return mbList
                r=choice(indicesN)
                sample=indicesN.pop(indicesN.index(r))
                mb.add(sample)
            mbList.append(mb)
        return mbList          #obj.mini_batches(#,bs=128) #: train_,train,val,test, we get lists of list of batches from here
    
    
    def calcG(self,ID):
        if ID>200:
            ID=200
        c=1
        somm=0
        while c<=ID:
            somm+=1/(math.log2(1+c))
            c+=1
        return somm

    def evalAcc(self,T,S,kneigh):
        T=T.detach().numpy().transpose()
        neigh=NearestNeighbors(n_neighbors=(kneigh+1),algorithm='ball_tree').fit(T)
        dist,ind=neigh.kneighbors(T)
        acc=[]
        for k in S:
            summ=0
            ideal=self.A[k,:].sum().item()  #gs
            den=self.calcG(ideal)
            c=1
            if den==0:#There is the problem of the distance for the people without neighboorhood
                continue
            for j in ind[k][1:]:
                if self.A[k][j]!=0:
                    summ+= 1/(math.log2(1+c))
                else:
                    continue
                c+=1
            summ/=den
            acc.append(summ)
        return acc


gs=GraphSAGE(X1,A1)
        


In [23]:
training=train_
testing=val
n_layer=1
batch_size=512 #This is the batch size used in the paper which insired artist similarity
mbb=gs.mini_batches(training,bs=batch_size)
num_epochs=1 #50

In [24]:
#With these lines of code we obtain the embedded space sample
start=time.time()
for epoch in range(num_epochs):
    print("Processing epoch n° ",epoch+1)
    num=int(len(training)/batch_size)+1     
    for k in range(len(mbb)):
        Ex=gs(mbb[k],n_layer)
        #TODO: Loss function, Optimizer
        name="C:\\Users\\Peppe\\OneDrive\\Desktop\\Università\\magistrale\\Neural_Networks\\MIR_project\\minibatches\\file"+str(k)+".pt"
        torch.save(Ex,name)

    for k in range(len(mbb)):
        name="C:\\Users\\Peppe\\OneDrive\\Desktop\\Università\\magistrale\\Neural_Networks\\MIR_project\\minibatches\\file"+str(k)+".pt"
        ex=torch.load(name)
        
        if k==0:
            t1=gs.tfunc(outf[-1],ex,mbb[k])

        else:
            t1=gs.tfunc2(outf[-1],ex,mbb[k],t1)
    print("Evaluating the epoch n° ",epoch+1)
    accL1=gs.evalAcc(t1[:,:training[-1]+1],set(training),KNN)
    t2=gs(set(testing),n_layer)
    t2=gs.tfunc2(outf[-1],t2,set(testing),t1) #We integrate the testing set to the previous training set
    accL2=gs.evalAcc(t2,set(testing),KNN)
    TestAcc=sum(accL1)/len(accL1)
    TrainAcc=sum(accL2)/len(accL2)
    print("Processesed epoch n° {}, \tTrain accuracy: {:.4f}, \tTest accuracy: {:.4f}".format((epoch+1),TrainAcc,TestAcc))
    print("done")
end=time.time()
print(end-start) #78 sec (1 graph layer), 132 sec (2 graph layers), 214 sec (3 graph layers).

Processing epoch n°  1
Evaluating the epoch n°  1
Processesed epoch n° 1, 	Train accuracy: 0.2365, 	Test accuracy: 0.3087
done
73.32841181755066


[0.0,
 0.0,
 0.5527849978433516,
 0.2478157200914361,
 0.5803167071934787,
 0.0,
 0.5505591812421098,
 0.3391602052736161,
 0.30260241349881345,
 0.5637884576902256,
 0.0,
 0.7738292444290639,
 0.0,
 0.41952291583572193,
 0.0,
 0.2906484777184987,
 0.5029917276850138,
 0.5857421563596757,
 0.8310649966911269,
 0.0,
 0.6131471927654584,
 0.5029917276850138,
 0.0,
 0.0,
 0.0,
 0.5030343053984369,
 0.5527849978433516,
 0.5148915307328632,
 0.9003541785812085,
 0.0,
 0.7606798040475855,
 0.7078800465594848,
 0.0,
 0.6744974905851283,
 0.0,
 0.3903800499921017,
 0.6744974905851283,
 0.0,
 0.46927872602275644,
 0.4565046246555299,
 0.38334277998463445,
 0.7475474611262282,
 0.2529427027676571,
 0.3903800499921017,
 0.6585765334200407,
 0.5637884576902256,
 0.0,
 0.39614437854106904,
 0.685530555364185,
 0.0,
 0.0,
 0.0,
 0.647939623894138,
 0.41253177989255346,
 0.3903800499921017,
 0.0,
 0.6704414151758369,
 0.6131471927654584,
 0.0,
 0.4483039899025303,
 0.5789533463717488,
 0.0,
 0.0,
 0.

In [20]:
# from sklearn.neighbors import NearestNeighbors
# npNeigh=t1.detach().numpy().transpose()
# nbrs=NearestNeighbors(n_neighbors=200,algorithm='ball_tree').fit(npNeigh[0:9021+1,:])

#distances,indices=nbrs.kneighbors(npNeigh[9022:10189,:])
#gr=nbrs.kneighbors_graph(npNeigh[0:9021+1,:]).toarray()
def calcG(ID):
    if ID>200:
        ID=200
    c=1
    somm=0
    while c<=ID:
        somm+=1/(math.log2(1+c))
        c+=1
    return somm
KNN=200
def evalAcc(T,S,kneigh):
    T=T.detach().numpy().transpose()
    neigh=NearestNeighbors(n_neighbors=(kneigh+1),algorithm='ball_tree').fit(T)
    dist,ind=neigh.kneighbors(T)
    print(dist)
    acc=[]
    for k in S:
        summ=0
        ideal=gs.A[k,:].sum().item()  #gs
        den=calcG(ideal)
        c=1
        if den==0:  #There is the problem of the distance for the people without neighboorhood
            continue
        for j in ind[k][1:]:
            if gs.A[k][j]!=0:
                summ+= 1/(math.log2(1+c))
            else:
                continue
            c+=1
        summ/=den
        acc.append(summ)
    return acc

#
tt=t1[:,:train_[-1]+1]
# from sklearn.neighbors import NearestNeighbors
# tt=tt.detach().numpy().transpose()
# nbrs=NearestNeighbors(n_neighbors=200+1,algorithm='ball_tree').fit(tt)

# distances,indices=nbrs.kneighbors(tt)    
evalAcc(tt,set(train_),200)

NameError: name 't1' is not defined

In [ ]:
def batch_size(rang,bs=32):
    m=[]
    while len(rang)!=0:
        l=set()
        while len(l)<bs:
            if len(rang)==0:
                m.append(l)
                return m
            else:
                r=choice(rang)
                sample=rang.pop(rang.index(r))

                l.add(sample)
            
        m.append(l)
        
    return m
c=batch_size(list(range(100)),bs=32)            

In [ ]:
c[-2]

In [ ]:
from random import randrange

print(randrange(-5, 5))

In [ ]:
def select(mat,row,col):
        col=sorted(list(col))
        ma=torch.zeros((mat.shape[0],len(col)))
        c=0
        if row==set():
            for k in col:
                ma[:,c]=mat[:,k]
                c+=1
            return [col,ma]
        else:
            row=sorted(list(row))
            return mat[row[0]:(row[-1]+1),col[0]:(col[-1]+1)]

def tfunc(n_feat,es,V):
    t=torch.zeros((n_feat,X.shape[1]))
    print(t.shape,es.shape)
    V=sorted(list(V))
    c=0
    for k in V:
        t[:,k]=es[:,c]
        c+=1
    return t
tfunc(2613,E,V[3])

In [ ]:
X=gs.X
V=gs.V

In [ ]:
def select(mat,row,col):
        col=sorted(list(col))
        
        c=0
        if row==set():
            ma=torch.zeros((mat.shape[0],len(col)))
            for k in col:
                ma[:,c]=mat[:,k]
                c+=1
            return ma
        else:
            row=torch.tensor(sorted(list(row)))
            col=torch.tensor(col)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
            
def tfunc(n_feat,es,V):
    t=torch.zeros((n_feat,X.shape[1]))
    l=select1(es,set(),V)
    V=l[0]
    es=l[1]
    c=0
    for k in V:
        t[:,k]=es[:,c]
        c+=1
    return t
select(A,{1,54,63},{56,876,900})